# 🐳 Diagnostic et Résolution des Erreurs de Build Docker

Ce notebook diagnostique l'erreur de construction d'image Docker rencontrée sur GitHub Actions lors de l'installation des packages Python optimisés.

## 🚨 Erreur rencontrée
```
ERROR: failed to build: failed to solve: process "/bin/bash -o pipefail -c pip3 install --upgrade pip setuptools wheel &&     pip3 install         einops safetensors jupyterlab ipywidgets         flash-attn --no-build-isolation         torch-audio-addons         triton         xformers &&     rm -rf /root/.cache/pip" did not complete successfully: exit code: 1
```

## 1. 📊 Analyser les logs d'erreur Docker

Les packages problématiques typiques dans ce type d'erreur :

In [ ]:
# Packages susceptibles de causer des problèmes de compilation
problematic_packages = {
    'flash-attn': {
        'compilation_required': True,
        'cuda_dependency': True,
        'memory_intensive': True,
        'common_issues': ['CUDA toolkit manquant', 'Mémoire insuffisante', 'GCC version incompatible']
    },
    'xformers': {
        'compilation_required': True,
        'cuda_dependency': True,
        'memory_intensive': True,
        'common_issues': ['PyTorch version incompatible', 'CUDA version mismatch']
    },
    'triton': {
        'compilation_required': True,
        'cuda_dependency': True,
        'memory_intensive': False,
        'common_issues': ['LLVM dependency', 'Architecture non supportée']
    },
    'torch-audio-addons': {
        'compilation_required': False,
        'cuda_dependency': False,
        'memory_intensive': False,
        'common_issues': ['Package inexistant ou obsolète']
    }
}

print("🔍 Analyse des packages problématiques:")
for package, info in problematic_packages.items():
    print(f"\n📦 {package}:")
    print(f"  • Compilation requise: {info['compilation_required']}")
    print(f"  • Dépendance CUDA: {info['cuda_dependency']}")
    print(f"  • Intensif en mémoire: {info['memory_intensive']}")
    print(f"  • Problèmes courants: {', '.join(info['common_issues'])}")

## 2. 🔍 Vérifier la compatibilité des dépendances Python

Analysons les versions et compatibilités requises :

In [ ]:
# Matrice de compatibilité PyTorch/CUDA pour les packages
compatibility_matrix = {
    'pytorch_2.8.0_cuda_12.9': {
        'flash-attn': {
            'supported': True,
            'version_range': '>=2.5.0',
            'notes': 'Nécessite CUDA 11.6+, compatible avec PyTorch 2.8'
        },
        'xformers': {
            'supported': True,
            'version_range': '>=0.0.23',
            'notes': 'Version compatible avec PyTorch 2.8 pas encore stable'
        },
        'triton': {
            'supported': True,
            'version_range': '>=2.1.0',
            'notes': 'Inclus avec PyTorch 2.8'
        },
        'torch-audio-addons': {
            'supported': False,
            'version_range': 'N/A',
            'notes': 'Package non trouvé - probablement obsolète'
        }
    }
}

print("📋 Matrice de compatibilité PyTorch 2.8.0 + CUDA 12.9:")
for package, info in compatibility_matrix['pytorch_2.8.0_cuda_12.9'].items():
    status = "✅" if info['supported'] else "❌"
    print(f"\n{status} {package}:")
    print(f"  • Version: {info['version_range']}")
    print(f"  • Notes: {info['notes']}")

## 3. 🧪 Tester l'installation des packages individuellement

Script pour identifier le package problématique :

In [ ]:
# Script de test d'installation séquentielle
test_script = """
#!/bin/bash
set -e

echo "🔧 Test d'installation séquentielle des packages..."

# Packages de base (peu de risques)
BASIC_PACKAGES="einops safetensors jupyterlab ipywidgets"

# Packages à risque (compilation CUDA)
RISKY_PACKAGES="flash-attn triton xformers"

# Package suspect
SUSPECT_PACKAGE="torch-audio-addons"

echo "📦 Installation des packages de base..."
pip install $BASIC_PACKAGES
echo "✅ Packages de base installés avec succès"

echo "🔥 Test du package suspect..."
if pip install $SUSPECT_PACKAGE; then
    echo "✅ $SUSPECT_PACKAGE installé avec succès"
else
    echo "❌ $SUSPECT_PACKAGE a échoué - probable cause de l'erreur"
fi

echo "⚡ Installation des packages à risque un par un..."
for package in $RISKY_PACKAGES; do
    echo "Installation de $package..."
    if pip install $package --no-build-isolation; then
        echo "✅ $package installé avec succès"
    else
        echo "❌ $package a échoué"
        exit 1
    fi
done
"""

print("📝 Script de test généré:")
print(test_script)

# Sauvegarder le script
with open('/Users/abscons/Documents/Docker/Runpod/Runpod_Github/test-packages.sh', 'w') as f:
    f.write(test_script)

print("\n💾 Script sauvegardé dans: test-packages.sh")

## 4. 🔧 Optimiser le Dockerfile pour la construction

Stratégies d'optimisation :

In [ ]:
# Générer un Dockerfile optimisé
optimized_dockerfile_section = """
# Installation des packages Python optimisés pour Blackwell - Version Robuste
RUN pip3 install --upgrade pip setuptools wheel && \\
    # Installation des packages de base (sans risque)
    pip3 install einops safetensors jupyterlab ipywidgets && \\
    # Installation de flash-attn avec gestion d'erreur
    (pip3 install flash-attn --no-build-isolation || \\
     echo "⚠️ flash-attn installation failed, continuing without it") && \\
    # Installation de xformers avec version spécifique
    (pip3 install xformers || \\
     pip3 install xformers==0.0.23.post1 || \\
     echo "⚠️ xformers installation failed, continuing without it") && \\
    # Triton est inclus avec PyTorch 2.8, installation optionnelle
    (pip3 install triton || echo "⚠️ triton installation failed, using PyTorch built-in version") && \\
    # Suppression du package problématique torch-audio-addons
    # Ce package semble obsolète ou inexistant
    rm -rf /root/.cache/pip
"""

print("🐳 Section Dockerfile optimisée:")
print(optimized_dockerfile_section)

# Alternative avec installation en deux étapes
two_stage_installation = """
# Alternative: Installation en deux étapes pour réduire les risques

# Étape 1: Packages de base
RUN pip3 install --upgrade pip setuptools wheel && \\
    pip3 install einops safetensors jupyterlab ipywidgets && \\
    rm -rf /root/.cache/pip

# Étape 2: Packages d'optimisation (avec gestion d'erreur)
RUN set +e && \\
    echo "Installation des optimisations GPU..." && \\
    pip3 install flash-attn --no-build-isolation && echo "✅ flash-attn installé" || echo "❌ flash-attn échoué" && \\
    pip3 install xformers && echo "✅ xformers installé" || echo "❌ xformers échoué" && \\
    pip3 install triton && echo "✅ triton installé" || echo "❌ triton échoué" && \\
    rm -rf /root/.cache/pip && \\
    set -e
"""

print("\n🔄 Alternative en deux étapes:")
print(two_stage_installation)

## 5. 🛡️ Implémenter des stratégies de fallback

Création de stratégies alternatives pour assurer le succès du build :

In [ ]:
# Stratégies de fallback pour chaque package
fallback_strategies = {
    'flash-attn': [
        'pip install flash-attn --no-build-isolation',
        'pip install flash-attn==2.5.8 --no-build-isolation',
        'pip install flash-attn --no-deps --no-build-isolation',
        'echo "Continuing without flash-attn"'
    ],
    'xformers': [
        'pip install xformers',
        'pip install xformers==0.0.23.post1',
        'pip install xformers --index-url https://download.pytorch.org/whl/cu121',
        'echo "Continuing without xformers"'
    ],
    'triton': [
        'pip install triton',
        'pip install triton==2.1.0',
        'echo "Using PyTorch built-in triton"'
    ],
    'torch-audio-addons': [
        'echo "Package torch-audio-addons obsolète, ignoré"'
    ]
}

# Générer un script de fallback
fallback_script = """
#!/bin/bash
# Script d'installation avec stratégies de fallback

install_with_fallback() {
    local package_name=$1
    shift
    local strategies=("$@")
    
    echo "📦 Installation de $package_name..."
    
    for strategy in "${strategies[@]}"; do
        echo "🔄 Tentative: $strategy"
        if eval $strategy; then
            echo "✅ $package_name installé avec succès"
            return 0
        else
            echo "❌ Stratégie échouée, tentative suivante..."
        fi
    done
    
    echo "⚠️ Toutes les stratégies ont échoué pour $package_name"
    return 1
}

# Installation des packages de base
pip install --upgrade pip setuptools wheel
pip install einops safetensors jupyterlab ipywidgets

# Installation avec fallback pour chaque package problématique
"""

for package, strategies in fallback_strategies.items():
    if package != 'torch-audio-addons':  # On ignore ce package obsolète
        strategy_array = ' '.join([f'"{s}"' for s in strategies])
        fallback_script += f"""
install_with_fallback "{package}" {strategy_array}
"""

fallback_script += """
# Nettoyage final
rm -rf /root/.cache/pip
echo "✅ Installation terminée"
"""

print("🛡️ Script de fallback généré:")
print(fallback_script)

# Sauvegarder le script
with open('/Users/abscons/Documents/Docker/Runpod/Runpod_Github/install-with-fallback.sh', 'w') as f:
    f.write(fallback_script)

print("\n💾 Script sauvegardé dans: install-with-fallback.sh")

## 🎯 Recommandations immédiates

### 1. **Problème identifié probable**: `torch-audio-addons`
Ce package semble **obsolète ou inexistant**. Il faut le supprimer.

### 2. **Solution immédiate**: Modifier le Dockerfile
Remplacer la section d'installation par une version robuste avec gestion d'erreurs.

### 3. **Test recommandé**: 
Utiliser le script `test-packages.sh` généré pour identifier précisément le package défaillant.

### 4. **Stratégie long terme**:
Implémenter le système de fallback pour assurer la robustesse du build.

In [ ]:
# Génération du correctif immédiat pour le Dockerfile
immediate_fix = """
# CORRECTIF IMMÉDIAT - Remplacer dans le Dockerfile principal:

# AVANT (version actuelle avec erreur):
RUN pip3 install --upgrade pip setuptools wheel && \\
    pip3 install \\
        einops safetensors jupyterlab ipywidgets \\
        flash-attn --no-build-isolation \\
        torch-audio-addons \\
        triton \\
        xformers && \\
    rm -rf /root/.cache/pip

# APRÈS (version corrigée):
RUN pip3 install --upgrade pip setuptools wheel && \\
    pip3 install \\
        einops safetensors jupyterlab ipywidgets && \\
    pip3 install flash-attn --no-build-isolation || echo "flash-attn installation failed" && \\
    pip3 install triton || echo "triton installation failed" && \\
    pip3 install xformers || echo "xformers installation failed" && \\
    rm -rf /root/.cache/pip

# CHANGEMENTS:
# ❌ Supprimé: torch-audio-addons (package obsolète/inexistant)
# ✅ Ajouté: Gestion d'erreur pour les packages d'optimisation
# ✅ Séparé: Installation en deux étapes pour isolation
"""

print("🔧 CORRECTIF IMMÉDIAT:")
print(immediate_fix)

# Sauvegarder le correctif
with open('/Users/abscons/Documents/Docker/Runpod/Runpod_Github/dockerfile-fix.md', 'w') as f:
    f.write(immediate_fix)

print("\n💾 Correctif sauvegardé dans: dockerfile-fix.md")
print("\n🚀 Prochaines étapes:")
print("1. Appliquer le correctif au Dockerfile")
print("2. Tester le build localement")
print("3. Commiter et pousser les changements")
print("4. Surveiller le build GitHub Actions")